<a href="https://colab.research.google.com/github/Emerson1993RC/Ferramentas/blob/main/C%C3%B3digos_de_organiza%C3%A7%C3%A3o_r%C3%A1pida_(Trabalho_PRO!).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00


In [2]:
import PyPDF2
import re

def extrair_resumo_producao_pdf(caminho_pdf: str):
    """
    Extrai apenas os dados da seção 'Resumo de produção' do PDF

    Args:
        caminho_pdf (str): Caminho para o arquivo PDF

    Returns:
        dict: Dicionário com registros identificados, não identificados e total
    """

    try:
        # Abrir e ler o PDF
        with open(caminho_pdf, 'rb') as arquivo:
            leitor_pdf = PyPDF2.PdfReader(arquivo)
            texto_completo = ""

            # Extrair texto de todas as páginas
            for pagina in leitor_pdf.pages:
                texto_completo += pagina.extract_text()

        # Limpar o texto removendo quebras de linha desnecessárias
        texto_limpo = limpar_texto(texto_completo)

        # Extrair apenas dados do Resumo de Produção
        resumo = {}

        # Procurar por "Registros identificados"
        match_identificados = re.search(r'Registros\s+identificados\s+(\d+)', texto_limpo, re.IGNORECASE)
        if match_identificados:
            resumo['registros_identificados'] = int(match_identificados.group(1))

        # Procurar por "Registros não identificados"
        match_nao_identificados = re.search(r'Registros\s+não\s+identificados\s+(\d+)', texto_limpo, re.IGNORECASE)
        if match_nao_identificados:
            resumo['registros_nao_identificados'] = int(match_nao_identificados.group(1))

        # Procurar pelo Total (linha que termina com "Total:")
        match_total = re.search(r'(\d+)\s*Total:', texto_limpo)
        if match_total:
            resumo['total'] = int(match_total.group(1))

        # Extrair dados adicionais específicos
        dados_adicionais = extrair_dados_especificos(texto_limpo)

        # Exibir resultados
        print("=" * 60)
        print("RESUMO DE PRODUÇÃO")
        print("=" * 60)

        for chave, valor in resumo.items():
            nome_campo = chave.replace('_', ' ').title()
            print(f"{nome_campo}: {valor}")

        print("\n" + "=" * 60)
        print("PROBLEMAS/CONDIÇÕES - CIAP2")
        print("=" * 60)

        ciap2 = dados_adicionais['ciap2']
        print(f"W78 - Gravidez: {ciap2.get('gravidez', 0)}")
        print(f"T90 - Diabetes Não Insulino-Dependente: {ciap2.get('diabetes_nao_insulino', 0)}")
        print(f"K86 - Hipertensão Sem Complicações: {ciap2.get('hipertensao_sem_complicacoes', 0)}")

        print("\n" + "=" * 60)
        print("PROBLEMAS/CONDIÇÕES - CID10")
        print("=" * 60)

        cid10 = dados_adicionais['cid10']
        print(f"I10 - Hipertensão Essencial (Primária): {cid10.get('hipertensao', 0)}")
        print(f"E10 - Diabetes Mellitus Insulino-Dependente: {cid10.get('diabetes_insulino_cid10', 0)}")

        print("\n" + "=" * 60)
        print("PROBLEMAS/CONDIÇÕES GERAIS")
        print("=" * 60)

        gerais = dados_adicionais['gerais']
        print(f"Puericultura: {gerais.get('puericultura', 0)}")

        print("\n" + "=" * 60)
        print("EXAMES SOLICITADOS")
        print("=" * 60)

        exames = dados_adicionais['exames']
        print(f"Hemoglobina Glicada (Solicitados): {exames.get('hemoglobina_glicada', 0)}")
        print(f"Ultrassonografia (Solicitados): {exames.get('ultrassonografia', 0)}")
        print(f"Eletrocardiograma (Solicitados): {exames.get('eletrocardiograma', 0)}")

        print("=" * 60)

        # Combinar todos os dados
        resultado_completo = {
            'resumo_producao': resumo,
            'ciap2': ciap2,
            'cid10': cid10,
            'gerais': gerais,
            'exames': exames
        }

        return resultado_completo

    except Exception as e:
        print(f"Erro ao processar o PDF: {str(e)}")
        return None


def limpar_texto(texto: str) -> str:
    """
    Limpa o texto removendo quebras de linha desnecessárias e normalizando espaços
    """
    # Remover quebras de linha dentro de palavras
    texto = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', texto)

    # Substituir múltiplas quebras de linha por espaço
    texto = re.sub(r'\n+', ' ', texto)

    # Normalizar espaços múltiplos
    texto = re.sub(r'\s+', ' ', texto)

    return texto.strip()


def extrair_dados_especificos(texto: str):
    """
    Extrai dados específicos das seções solicitadas
    """
    dados = {
        'ciap2': {},
        'cid10': {},
        'gerais': {},
        'exames': {}
    }

    # ============================================
    # 1. CIAP2 - Gravidez, Diabetes e Hipertensão
    # ============================================

    # W78 - GRAVIDEZ
    match_gravidez = re.search(r'W78\s*-\s*GRAVIDEZ\s+(\d+)', texto, re.IGNORECASE)
    if match_gravidez:
        dados['ciap2']['gravidez'] = int(match_gravidez.group(1))

    # T90 - DIABETES NÃO INSULINO-DEPENDENTE
    match_diabetes_nao_insulino = re.search(r'T90\s*-\s*DIABETES\s+NÃO\s+INSULINO\s*-?\s*DEPENDENTE\s+(\d+)', texto, re.IGNORECASE)
    if match_diabetes_nao_insulino:
        dados['ciap2']['diabetes_nao_insulino'] = int(match_diabetes_nao_insulino.group(1))

    # K86 - HIPERTENSÃO SEM COMPLICAÇÕES
    match_hipertensao_ciap2 = re.search(r'K86\s*-\s*HIPERTENSÃO\s+SEM\s+COMPLICAÇÕES\s+(\d+)', texto, re.IGNORECASE)
    if match_hipertensao_ciap2:
        dados['ciap2']['hipertensao_sem_complicacoes'] = int(match_hipertensao_ciap2.group(1))

    # =====================================
    # 2. CID10 - Hipertensão e Diabetes
    # =====================================

    # I10 - HIPERTENSÃO ESSENCIAL (PRIMÁRIA)
    match_hipertensao = re.search(r'I10\s*-\s*HIPERTENSÃO\s+ESSENCIAL\s*\(?PRIMÁRIA\)?\s+(\d+)', texto, re.IGNORECASE)
    if match_hipertensao:
        dados['cid10']['hipertensao'] = int(match_hipertensao.group(1))

    # E10 - DIABETES MELLITUS INSULINO-DEPENDENTE (múltiplos padrões)
    match_diabetes_insulino_cid10 = None

    # Padrão 1: Formato completo
    match_diabetes_insulino_cid10 = re.search(r'E10\s*-\s*DIABETES\s+MELLITUS\s+INSULINO\s*-?\s*DEPENDENTE\s+(\d+)', texto, re.IGNORECASE)

    # Padrão 2: Formato mais simples (caso tenha quebras ou espaçamento diferente)
    if not match_diabetes_insulino_cid10:
        match_diabetes_insulino_cid10 = re.search(r'E10.*?DIABETES.*?MELLITUS.*?INSULINO.*?DEPENDENTE\s+(\d+)', texto, re.IGNORECASE | re.DOTALL)

    # Padrão 3: Procurar por E10 seguido de número (último recurso)
    if not match_diabetes_insulino_cid10:
        # Primeiro encontrar todas as ocorrências de E10
        e10_matches = list(re.finditer(r'E10\s*-[^0-9]*?(\d+)', texto, re.IGNORECASE))
        if e10_matches:
            # Pegar a última ocorrência (geralmente a mais específica)
            match_diabetes_insulino_cid10 = e10_matches[-1]

    if match_diabetes_insulino_cid10:
        dados['cid10']['diabetes_insulino_cid10'] = int(match_diabetes_insulino_cid10.group(1))

    # ==========================================
    # 3. Problemas/Condições Gerais
    # ==========================================

    # Puericultura
    match_puericultura = re.search(r'Puericultura\s+(\d+)', texto, re.IGNORECASE)
    if match_puericultura:
        dados['gerais']['puericultura'] = int(match_puericultura.group(1))

    # ==========================================
    # 4. Exames Solicitados
    # ==========================================

    # Hemoglobina glicada (apenas solicitados)
    match_hemoglobina = re.search(r'Hemoglobina\s+glicada\s+(\d+)\s+\d+', texto, re.IGNORECASE)
    if match_hemoglobina:
        dados['exames']['hemoglobina_glicada'] = int(match_hemoglobina.group(1))

    # Eletrocardiograma (apenas solicitados)
    match_eletro = re.search(r'Eletrocardiograma\s+(\d+)\s+\d+', texto, re.IGNORECASE)
    if match_eletro:
        dados['exames']['eletrocardiograma'] = int(match_eletro.group(1))

    # Ultrassonografia (somar todas as ultrassonografias solicitadas)
    ultrassom_total = 0

    # Ultrassonografia obstétrica
    ultrassom_obst = re.search(r'Ultrassonografia\s+obstétrica\s+(\d+)\s+\d+', texto, re.IGNORECASE)
    if ultrassom_obst:
        ultrassom_total += int(ultrassom_obst.group(1))

    # Outras ultrassonografias (procurar por códigos SIGTAP)
    ultrassom_matches = re.findall(r'ULTRASSONOGRAFIA[^0-9\n]+(\d+)\s+\d+', texto, re.IGNORECASE)
    for match in ultrassom_matches:
        ultrassom_total += int(match)

    dados['exames']['ultrassonografia'] = ultrassom_total

    return dados


def processar_pdf(caminho_pdf):
    """
    Função principal simplificada

    Usage no Google Colab:
    1. Faça upload do PDF
    2. Execute: resultado = processar_pdf('/content/nome_do_arquivo.pdf')
    """
    return extrair_resumo_producao_pdf(caminho_pdf)


def debug_ciap2_especifico(caminho_pdf):
    """
    Debug específico para códigos CIAP2
    """
    try:
        with open(caminho_pdf, 'rb') as arquivo:
            leitor_pdf = PyPDF2.PdfReader(arquivo)
            texto_completo = ""

            for pagina in leitor_pdf.pages:
                texto_completo += pagina.extract_text()

        print("=== DEBUG CIAP2 ESPECÍFICO ===")

        # Códigos CIAP2 para procurar
        codigos_ciap2 = ['W78', 'T90', 'K86']

        for codigo in codigos_ciap2:
            print(f"\n{codigo}:")
            matches = list(re.finditer(f'{codigo}[^0-9]*?(\\d+)', texto_completo, re.IGNORECASE | re.DOTALL))
            for i, match in enumerate(matches):
                print(f"  Match {i+1}: {repr(match.group())}")
                contexto = texto_completo[max(0, match.start()-50):match.end()+30]
                print(f"  Contexto: {repr(contexto)}")
                print(f"  Número: {match.group(1)}")
                print("  " + "-" * 40)

        # Texto limpo
        texto_limpo = limpar_texto(texto_completo)
        print(f"\n=== TEXTO LIMPO - CIAP2 ===")
        for codigo in codigos_ciap2:
            matches = list(re.finditer(f'{codigo}[^0-9]*?(\\d+)', texto_limpo, re.IGNORECASE))
            if matches:
                print(f"{codigo}: {matches[-1].group(1)}")

    except Exception as e:
        print(f"Erro no debug CIAP2: {e}")


def debug_e10_especifico(caminho_pdf):
    """
    Debug específico para encontrar E10 - DIABETES MELLITUS INSULINO-DEPENDENTE
    """
    try:
        with open(caminho_pdf, 'rb') as arquivo:
            leitor_pdf = PyPDF2.PdfReader(arquivo)
            texto_completo = ""

            for pagina in leitor_pdf.pages:
                texto_completo += pagina.extract_text()

        print("=== DEBUG E10 ESPECÍFICO ===")

        # 1. Procurar todas as ocorrências de E10
        print("1. Todas as ocorrências de E10:")
        e10_todas = list(re.finditer(r'E10[^0-9]*?(\d+)', texto_completo, re.IGNORECASE | re.DOTALL))
        for i, match in enumerate(e10_todas):
            print(f"E10 Match {i+1}: {repr(match.group())}")
            contexto = texto_completo[max(0, match.start()-30):match.end()+30]
            print(f"Contexto: {repr(contexto)}")
            print(f"Número extraído: {match.group(1)}")
            print("-" * 50)

        # 2. Procurar especificamente por DIABETES MELLITUS INSULINO
        print("\n2. Procurando DIABETES MELLITUS INSULINO:")
        diabetes_matches = list(re.finditer(r'DIABETES\s+MELLITUS\s+INSULINO[^0-9]*?(\d+)', texto_completo, re.IGNORECASE | re.DOTALL))
        for i, match in enumerate(diabetes_matches):
            print(f"Diabetes Match {i+1}: {repr(match.group())}")
            contexto = texto_completo[max(0, match.start()-50):match.end()+30]
            print(f"Contexto: {repr(contexto)}")
            print(f"Número extraído: {match.group(1)}")
            print("-" * 50)

        # 3. Texto limpo
        texto_limpo = limpar_texto(texto_completo)
        print("\n3. No texto limpo - E10:")
        e10_limpo = list(re.finditer(r'E10[^0-9]*?(\d+)', texto_limpo, re.IGNORECASE))
        for i, match in enumerate(e10_limpo):
            print(f"E10 Limpo {i+1}: {repr(match.group())}")
            contexto = texto_limpo[max(0, match.start()-30):match.end()+30]
            print(f"Contexto: {repr(contexto)}")
            print(f"Número extraído: {match.group(1)}")
            print("-" * 50)

    except Exception as e:
        print(f"Erro no debug: {e}")


def debug_texto(caminho_pdf):
    """
    Função para debug - mostra como o texto está sendo extraído
    """
    try:
        with open(caminho_pdf, 'rb') as arquivo:
            leitor_pdf = PyPDF2.PdfReader(arquivo)
            texto_completo = ""

            for pagina in leitor_pdf.pages:
                texto_completo += pagina.extract_text()

        # Procurar especificamente por E10
        print("Procurando por E10 no texto original:")
        matches_e10 = re.finditer(r'E10.*?(\d+)', texto_completo, re.IGNORECASE | re.DOTALL)
        for i, match in enumerate(matches_e10):
            print(f"Match {i+1}: {repr(match.group())}")
            print(f"Contexto: {repr(texto_completo[max(0, match.start()-50):match.end()+50])}")
            print("-" * 50)

        # Verificar texto limpo
        texto_limpo = limpar_texto(texto_completo)
        print("\nProcurando por E10 no texto limpo:")
        matches_e10_limpo = re.finditer(r'E10.*?(\d+)', texto_limpo, re.IGNORECASE)
        for i, match in enumerate(matches_e10_limpo):
            print(f"Match {i+1}: {repr(match.group())}")
            print("-" * 50)

    except Exception as e:
        print(f"Erro: {e}")


# Exemplo de uso
if __name__ == "__main__":
    print("🔧 CÓDIGO ATUALIZADO COM K86!")
    print("=" * 50)
    print("Funções disponíveis:")
    print("  resultado = processar_pdf('SILVIO.pdf')")
    print("  debug_ciap2_especifico('SILVIO.pdf')       # Debug para W78, T90, K86")
    print("  debug_e10_especifico('SILVIO.pdf')         # Debug para E10")
    print("  debug_texto('SILVIO.pdf')                  # Debug geral")
    print("=" * 50)

🔧 CÓDIGO ATUALIZADO COM K86!
Funções disponíveis:
  resultado = processar_pdf('SILVIO.pdf')
  debug_ciap2_especifico('SILVIO.pdf')       # Debug para W78, T90, K86
  debug_e10_especifico('SILVIO.pdf')         # Debug para E10
  debug_texto('SILVIO.pdf')                  # Debug geral


In [7]:
resultado = processar_pdf('/content/silvio 1.pdf')

RESUMO DE PRODUÇÃO - MÉDICO
Registros Identificados: 63
Registros Nao Identificados: 0
Total: 63

PROBLEMAS/CONDIÇÕES - CIAP2
W78 - Gravidez: 4
T90 - Diabetes Não Insulino-Dependente: 3
K86 - Hipertensão Sem Complicações: 17

PROBLEMAS/CONDIÇÕES - CID10
I10 - Hipertensão Essencial (Primária): 0
E10 - Diabetes Mellitus Insulino-Dependente: 1

PROBLEMAS/CONDIÇÕES GERAIS
Puericultura: 0
Gestantes: 4
Pacientes com Necessidades Especiais: 0

EXAMES SOLICITADOS
Hemoglobina Glicada (Solicitados): 4
Ultrassonografia (Solicitados): 1
Eletrocardiograma (Solicitados): 1


In [4]:
!pip install PyPDF2 pandas matplotlib seaborn openpyxl

In [19]:
import PyPDF2
import re
from pathlib import Path

def limpar_texto(texto: str) -> str:
    """Remove quebras de linha desnecessárias e normaliza espaços"""
    texto = texto.replace('\n', ' ')
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()


def extrair_dados_especificos(texto: str):
    """
    Extrai dados específicos das seções solicitadas (médicos e odontológicos)
    """
    dados = {
        'ciap2': {},
        'cid10': {},
        'gerais': {},
        'exames': {},
        'procedimentos': {}
    }

    # ============================================
    # 1. CIAP2 - Gravidez, Diabetes e Hipertensão
    # ============================================

    # W78 - GRAVIDEZ
    match_gravidez = re.search(r'W78\s*-\s*GRAVIDEZ\s+(\d+)', texto, re.IGNORECASE)
    if match_gravidez:
        dados['ciap2']['gravidez'] = int(match_gravidez.group(1))

    # T90 - DIABETES NÃO INSULINO-DEPENDENTE
    match_diabetes_nao_insulino = re.search(r'T90\s*-\s*DIABETES\s+NÃO\s+INSULINO\s*-?\s*DEPENDENTE\s+(\d+)', texto, re.IGNORECASE)
    if match_diabetes_nao_insulino:
        dados['ciap2']['diabetes_nao_insulino'] = int(match_diabetes_nao_insulino.group(1))

    # K86 - HIPERTENSÃO SEM COMPLICAÇÕES
    match_hipertensao_ciap2 = re.search(r'K86\s*-\s*HIPERTENSÃO\s+SEM\s+COMPLICAÇÕES\s+(\d+)', texto, re.IGNORECASE)
    if match_hipertensao_ciap2:
        dados['ciap2']['hipertensao_sem_complicacoes'] = int(match_hipertensao_ciap2.group(1))

    # =====================================
    # 2. CID10 - Hipertensão e Diabetes
    # =====================================

    # I10 - HIPERTENSÃO ESSENCIAL (PRIMÁRIA)
    match_hipertensao = re.search(r'I10\s*-\s*HIPERTENSÃO\s+ESSENCIAL\s*\(?PRIMÁRIA\)?\s+(\d+)', texto, re.IGNORECASE)
    if match_hipertensao:
        dados['cid10']['hipertensao'] = int(match_hipertensao.group(1))

    # E10 - DIABETES MELLITUS INSULINO-DEPENDENTE
    match_diabetes_insulino_cid10 = None
    match_diabetes_insulino_cid10 = re.search(r'E10\s*-\s*DIABETES\s+MELLITUS\s+INSULINO\s*-?\s*DEPENDENTE\s+(\d+)', texto, re.IGNORECASE)

    if not match_diabetes_insulino_cid10:
        match_diabetes_insulino_cid10 = re.search(r'E10.*?DIABETES.*?MELLITUS.*?INSULINO.*?DEPENDENTE\s+(\d+)', texto, re.IGNORECASE | re.DOTALL)

    if not match_diabetes_insulino_cid10:
        e10_matches = list(re.finditer(r'E10\s*-[^0-9]*?(\d+)', texto, re.IGNORECASE))
        if e10_matches:
            match_diabetes_insulino_cid10 = e10_matches[-1]

    if match_diabetes_insulino_cid10:
        dados['cid10']['diabetes_insulino_cid10'] = int(match_diabetes_insulino_cid10.group(1))

    # ==========================================
    # 3. Problemas/Condições Gerais
    # ==========================================

    # PUERICULTURA - Padrão correto: ABP004 - PUERICULTURA
    match_puericultura = re.search(r'ABP004\s*-\s*PUERICULTURA\s+(\d+)', texto, re.IGNORECASE)
    if match_puericultura:
        dados['gerais']['puericultura'] = int(match_puericultura.group(1))
    else:
        # Tentar padrão alternativo
        match_puericultura_alt = re.search(r'Puericultura\s+(\d+)', texto, re.IGNORECASE)
        if match_puericultura_alt:
            dados['gerais']['puericultura'] = int(match_puericultura_alt.group(1))

    # Gestante
    match_gestante = re.search(r'Gestante\s+(\d+)', texto, re.IGNORECASE)
    if match_gestante:
        dados['gerais']['gestante'] = int(match_gestante.group(1))
        if 'gravidez' not in dados['ciap2']:
            dados['ciap2']['gravidez'] = int(match_gestante.group(1))

    # Paciente com necessidades especiais
    match_necessidades_especiais = re.search(r'Paciente\s+com\s+necessidades\s+especiais\s+(\d+)', texto, re.IGNORECASE)
    if match_necessidades_especiais:
        dados['gerais']['necessidades_especiais'] = int(match_necessidades_especiais.group(1))

    # Visitas Domiciliares
    match_visita_domiciliar = re.search(r'0101030010\s*-\s*VISITA\s+DOMICILIAR\s+POR\s+PROFISSIONAL\s+DE\s+NÍVEL\s+MÉDIO\s+(\d+)', texto, re.IGNORECASE)
    if match_visita_domiciliar:
        dados['gerais']['visitas_domiciliares'] = int(match_visita_domiciliar.group(1))
    else:
        # Tentar pelo Local de atendimento - Domicílio
        match_domicilio = re.search(r'Domicílio\s+(\d+)', texto, re.IGNORECASE)
        if match_domicilio:
            dados['gerais']['visitas_domiciliares'] = int(match_domicilio.group(1))

    # ==========================================
    # 4. Procedimentos SIGTAP
    # ==========================================

    # Aferição de Pressão Arterial
    match_afericao_pressao = re.search(r'0301100039\s*-\s*AFERIÇÃO\s+DE\s+PRESSÃO\s+ARTERIAL\s+(\d+)', texto, re.IGNORECASE)
    if match_afericao_pressao:
        dados['procedimentos']['afericao_pressao_arterial'] = int(match_afericao_pressao.group(1))

    # Total de Outros Procedimentos SIGTAP
    match_total_sigtap = re.search(r'Outros\s+procedimentos\s*\(SIGTAP\).*?(\d+)\s*Total:', texto, re.IGNORECASE | re.DOTALL)
    if match_total_sigtap:
        dados['procedimentos']['total_sigtap'] = int(match_total_sigtap.group(1))

    # ==========================================
    # 5. Exames Solicitados (apenas coluna Solicitado)
    # ==========================================

    # Hemoglobina glicada (apenas solicitados)
    match_hemoglobina = re.search(r'Hemoglobina\s+glicada\s+(\d+)\s+\d+', texto, re.IGNORECASE)
    if match_hemoglobina:
        dados['exames']['hemoglobina_glicada_solicitado'] = int(match_hemoglobina.group(1))

    # Eletrocardiograma (apenas solicitados)
    match_eletro = re.search(r'Eletrocardiograma\s+(\d+)\s+\d+', texto, re.IGNORECASE)
    if match_eletro:
        dados['exames']['eletrocardiograma_solicitado'] = int(match_eletro.group(1))

    # Ultrassonografia (somar todas as ultrassonografias solicitadas)
    ultrassom_total = 0
    ultrassom_obst = re.search(r'Ultrassonografia\s+obstétrica\s+(\d+)\s+\d+', texto, re.IGNORECASE)
    if ultrassom_obst:
        ultrassom_total += int(ultrassom_obst.group(1))

    ultrassom_matches = re.findall(r'ULTRASSONOGRAFIA[^0-9\n]+(\d+)\s+\d+', texto, re.IGNORECASE)
    for match in ultrassom_matches:
        ultrassom_total += int(match)

    dados['exames']['ultrassonografia_solicitado'] = ultrassom_total

    # Total de Exames Solicitados - Seção principal
    total_exames_principais = 0
    match_total_exames_principais = re.search(r'Exames\s+solicitados\s+e\s+avaliados.*?(\d+)\s*Total:\s+\d+', texto, re.IGNORECASE | re.DOTALL)
    if match_total_exames_principais:
        total_exames_principais = int(match_total_exames_principais.group(1))

    # Total de Outros Exames SIGTAP - Seção "Outros exames solicitados e avaliados (código do SIGTAP)"
    total_outros_exames_sigtap = 0
    match_outros_exames = re.search(r'Outros\s+exames\s+solicitados\s+e\s+avaliados\s*\(código\s+do\s+SIGTAP\).*?(\d+)\s*Total:\s+\d+', texto, re.IGNORECASE | re.DOTALL)
    if match_outros_exames:
        total_outros_exames_sigtap = int(match_outros_exames.group(1))

    # Armazenar os valores separados e o total
    dados['exames']['exames_principais_solicitados'] = total_exames_principais
    dados['exames']['outros_exames_sigtap_solicitados'] = total_outros_exames_sigtap
    dados['exames']['total_exames_solicitados'] = total_exames_principais + total_outros_exames_sigtap

    return dados


def extrair_resumo_producao_pdf(caminho_pdf: str):
    """
    Extrai dados do PDF (relatórios médicos e odontológicos)

    Args:
        caminho_pdf (str): Caminho para o arquivo PDF

    Returns:
        dict: Dicionário com todos os dados extraídos
    """
    try:
        with open(caminho_pdf, 'rb') as arquivo:
            leitor_pdf = PyPDF2.PdfReader(arquivo)
            texto_completo = ""

            for pagina in leitor_pdf.pages:
                texto_completo += pagina.extract_text()

        texto_limpo = limpar_texto(texto_completo)

        # Determinar tipo de relatório
        tipo_relatorio = "médico"
        if "relatório de atendimento odontológico" in texto_limpo.lower():
            tipo_relatorio = "odontológico"
        elif "relatório de procedimentos individualizados" in texto_limpo.lower():
            tipo_relatorio = "procedimentos"

        # Extrair profissional
        profissional = "Não identificado"
        match_profissional = re.search(r'Profissional:\s*\[[\d\s,]+\]\s*-\s*([^|]+)', texto_limpo, re.IGNORECASE)
        if match_profissional:
            profissional = match_profissional.group(1).strip()

        # Extrair CBO
        cbo = "Não identificado"
        match_cbo = re.search(r'CBO:\s*([\d]+)\s*-\s*([^|]+)', texto_limpo, re.IGNORECASE)
        if match_cbo:
            cbo = f"{match_cbo.group(1)} - {match_cbo.group(2).strip()}"

        # Extrair dados do Resumo de Produção
        resumo = {}

        match_identificados = re.search(r'Registros\s+identificados\s+(\d+)', texto_limpo, re.IGNORECASE)
        if match_identificados:
            resumo['registros_identificados'] = int(match_identificados.group(1))

        match_nao_identificados = re.search(r'Registros\s+não\s+identificados\s+(\d+)', texto_limpo, re.IGNORECASE)
        if match_nao_identificados:
            resumo['registros_nao_identificados'] = int(match_nao_identificados.group(1))

        match_total = re.search(r'(\d+)\s*Total:', texto_limpo)
        if match_total:
            resumo['total'] = int(match_total.group(1))

        # Extrair dados específicos
        dados_adicionais = extrair_dados_especificos(texto_limpo)

        # Exibir resultados
        print("=" * 70)
        print(f"ARQUIVO: {Path(caminho_pdf).name}")
        print(f"PROFISSIONAL: {profissional}")
        print(f"CBO: {cbo}")
        print(f"TIPO: {tipo_relatorio.upper()}")
        print("=" * 70)

        for chave, valor in resumo.items():
            nome_campo = chave.replace('_', ' ').title()
            print(f"{nome_campo}: {valor}")

        ciap2 = dados_adicionais['ciap2']
        if any(ciap2.values()):
            print("\n" + "=" * 70)
            print("PROBLEMAS/CONDIÇÕES - CIAP2")
            print("=" * 70)
            print(f"W78 - Gravidez: {ciap2.get('gravidez', 0)}")
            print(f"T90 - Diabetes Não Insulino-Dependente: {ciap2.get('diabetes_nao_insulino', 0)}")
            print(f"K86 - Hipertensão Sem Complicações: {ciap2.get('hipertensao_sem_complicacoes', 0)}")

        cid10 = dados_adicionais['cid10']
        if any(cid10.values()):
            print("\n" + "=" * 70)
            print("PROBLEMAS/CONDIÇÕES - CID10")
            print("=" * 70)
            print(f"I10 - Hipertensão Essencial (Primária): {cid10.get('hipertensao', 0)}")
            print(f"E10 - Diabetes Mellitus Insulino-Dependente: {cid10.get('diabetes_insulino_cid10', 0)}")

        gerais = dados_adicionais['gerais']
        if any(gerais.values()) or ciap2.get('gravidez', 0) > 0:
            print("\n" + "=" * 70)
            print("PROBLEMAS/CONDIÇÕES GERAIS")
            print("=" * 70)
            print(f"Puericultura (ABP004): {gerais.get('puericultura', 0)}")
            print(f"Gestantes: {gerais.get('gestante', ciap2.get('gravidez', 0))}")
            print(f"Pacientes com Necessidades Especiais: {gerais.get('necessidades_especiais', 0)}")
            print(f"Visitas Domiciliares: {gerais.get('visitas_domiciliares', 0)}")

        procedimentos = dados_adicionais['procedimentos']
        if any(procedimentos.values()):
            print("\n" + "=" * 70)
            print("PROCEDIMENTOS")
            print("=" * 70)
            print(f"Aferição de Pressão Arterial (0301100039): {procedimentos.get('afericao_pressao_arterial', 0)}")
            print(f"Total Outros Procedimentos SIGTAP: {procedimentos.get('total_sigtap', 0)}")

        exames = dados_adicionais['exames']
        if any(exames.values()):
            print("\n" + "=" * 70)
            print("EXAMES SOLICITADOS")
            print("=" * 70)
            print(f"Hemoglobina Glicada: {exames.get('hemoglobina_glicada_solicitado', 0)}")
            print(f"Ultrassonografia: {exames.get('ultrassonografia_solicitado', 0)}")
            print(f"Eletrocardiograma: {exames.get('eletrocardiograma_solicitado', 0)}")
            print(f"\nExames Principais Solicitados: {exames.get('exames_principais_solicitados', 0)}")
            print(f"Outros Exames SIGTAP Solicitados: {exames.get('outros_exames_sigtap_solicitados', 0)}")
            print(f"─" * 70)
            print(f"TOTAL DE EXAMES SOLICITADOS: {exames.get('total_exames_solicitados', 0)}")
            print(f"─" * 70)

        print("=" * 70 + "\n")

        # Combinar todos os dados
        resultado_completo = {
            'arquivo': Path(caminho_pdf).name,
            'profissional': profissional,
            'cbo': cbo,
            'tipo_relatorio': tipo_relatorio,
            'resumo_producao': resumo,
            'ciap2': ciap2,
            'cid10': cid10,
            'gerais': gerais,
            'procedimentos': procedimentos,
            'exames': exames
        }

        return resultado_completo

    except Exception as e:
        print(f"Erro ao processar o PDF {caminho_pdf}: {str(e)}")
        return None


def processar_multiplos_pdfs(pasta_pdfs=None, extensao='.pdf'):
    """
    Processa múltiplos arquivos PDF em uma pasta

    Args:
        pasta_pdfs (str): Caminho da pasta com os PDFs. Se None, usa a pasta atual
        extensao (str): Extensão dos arquivos a processar (padrão: '.pdf')

    Returns:
        list: Lista com resultados de todos os PDFs processados
    """
    if pasta_pdfs is None:
        pasta_pdfs = '.'

    pasta = Path(pasta_pdfs)

    # Encontrar todos os arquivos PDF
    arquivos_pdf = list(pasta.glob(f'*{extensao}'))

    if not arquivos_pdf:
        print(f"Nenhum arquivo {extensao} encontrado em {pasta_pdfs}")
        return []

    print(f"\n{'='*70}")
    print(f"Encontrados {len(arquivos_pdf)} arquivo(s) PDF")
    print(f"{'='*70}\n")

    resultados = []

    for arquivo_pdf in arquivos_pdf:
        print(f"Processando: {arquivo_pdf.name}...")
        resultado = extrair_resumo_producao_pdf(str(arquivo_pdf))

        if resultado:
            resultados.append(resultado)

    # Criar resumo consolidado
    if resultados:
        print("\n" + "="*70)
        print("RESUMO CONSOLIDADO POR PROFISSIONAL")
        print("="*70)

        # Agrupar por profissional
        profissionais = {}
        for r in resultados:
            prof = r['profissional']
            if prof not in profissionais:
                profissionais[prof] = []
            profissionais[prof].append(r)

        for prof, docs in profissionais.items():
            print(f"\n{prof}:")
            print(f"  - Total de documentos: {len(docs)}")
            total_registros = sum(d['resumo_producao'].get('registros_identificados', 0) for d in docs)
            print(f"  - Total de registros identificados: {total_registros}")
            total_visitas = sum(d['gerais'].get('visitas_domiciliares', 0) for d in docs)
            if total_visitas > 0:
                print(f"  - Total de visitas domiciliares: {total_visitas}")
            total_exames = sum(d['exames'].get('total_exames_solicitados', 0) for d in docs)
            if total_exames > 0:
                print(f"  - Total de exames solicitados: {total_exames}")

        print("\n" + "="*70)
        print(f"Total de arquivos processados: {len(resultados)}")
        print(f"Relatórios médicos: {sum(1 for r in resultados if r['tipo_relatorio'] == 'médico')}")
        print(f"Relatórios odontológicos: {sum(1 for r in resultados if r['tipo_relatorio'] == 'odontológico')}")
        print(f"Relatórios de procedimentos: {sum(1 for r in resultados if r['tipo_relatorio'] == 'procedimentos')}")
        print("="*70 + "\n")

    return resultados


# Exemplo de uso
if __name__ == "__main__":
    # Processar todos os PDFs da pasta atual
    resultados = processar_multiplos_pdfs()


Encontrados 4 arquivo(s) PDF

Processando: silvio 1.pdf...
ARQUIVO: silvio 1.pdf
PROFISSIONAL: Silvio Cesar Aragao Goncalves
CBO: 225142 - MÉDICO DA ESTRATÉGIA DE SAÚDE DA FAMÍLIA
TIPO: MÉDICO
Registros Identificados: 63
Registros Nao Identificados: 0
Total: 63

PROBLEMAS/CONDIÇÕES - CIAP2
W78 - Gravidez: 4
T90 - Diabetes Não Insulino-Dependente: 3
K86 - Hipertensão Sem Complicações: 17

PROBLEMAS/CONDIÇÕES - CID10
I10 - Hipertensão Essencial (Primária): 0
E10 - Diabetes Mellitus Insulino-Dependente: 1

PROBLEMAS/CONDIÇÕES GERAIS
Puericultura (ABP004): 2
Gestantes: 4
Pacientes com Necessidades Especiais: 0
Visitas Domiciliares: 14

EXAMES SOLICITADOS
Hemoglobina Glicada: 4
Ultrassonografia: 1
Eletrocardiograma: 1

Exames Principais Solicitados: 30
Outros Exames SIGTAP Solicitados: 32
──────────────────────────────────────────────────────────────────────
TOTAL DE EXAMES SOLICITADOS: 62
──────────────────────────────────────────────────────────────────────

Processando: manoel (1).pdf..

In [17]:
resultados = processar_multiplos_pdfs()


Encontrados 4 arquivo(s) PDF

Processando: silvio 1.pdf...
ARQUIVO: silvio 1.pdf
TIPO: MÉDICO
Registros Identificados: 63
Registros Nao Identificados: 0
Total: 63

PROBLEMAS/CONDIÇÕES - CIAP2
W78 - Gravidez: 4
T90 - Diabetes Não Insulino-Dependente: 3
K86 - Hipertensão Sem Complicações: 17

PROBLEMAS/CONDIÇÕES - CID10
I10 - Hipertensão Essencial (Primária): 0
E10 - Diabetes Mellitus Insulino-Dependente: 1

PROBLEMAS/CONDIÇÕES GERAIS
Puericultura: 0
Gestantes: 4
Pacientes com Necessidades Especiais: 0

EXAMES SOLICITADOS
Hemoglobina Glicada: 4
Ultrassonografia: 1
Eletrocardiograma: 1

Processando: manoel (1).pdf...
ARQUIVO: manoel (1).pdf
TIPO: ODONTOLÓGICO
Registros Identificados: 101
Registros Nao Identificados: 0
Total: 101

PROBLEMAS/CONDIÇÕES - CIAP2
W78 - Gravidez: 4
T90 - Diabetes Não Insulino-Dependente: 0
K86 - Hipertensão Sem Complicações: 0

PROBLEMAS/CONDIÇÕES GERAIS
Puericultura: 0
Gestantes: 4
Pacientes com Necessidades Especiais: 28

Processando: emerson1.pdf...
ARQUIVO:

In [10]:
from google.colab import files
import PyPDF2
import re
import os

def limpar_texto(texto: str) -> str:
    """
    Limpa o texto removendo quebras de linha desnecessárias e normalizando espaços
    """
    # Remover quebras de linha dentro de palavras
    texto = re.sub(r'(\w)-\s*\n\s*(\w)', r'\1\2', texto)

    # Substituir múltiplas quebras de linha por espaço
    texto = re.sub(r'\n+', ' ', texto)

    # Normalizar espaços múltiplos
    texto = re.sub(r'\s+', ' ', texto)

    return texto.strip()

def extrair_faixas_etarias_pdf(caminho_pdf: str):
    """
    Extrai dados específicos das faixas etárias de 10 a 14 anos e 15 a 19 anos,
    separando por sexo.

    Args:
        caminho_pdf (str): Caminho para o arquivo PDF.

    Returns:
        dict: Dicionário com os totais de pacientes por faixa etária e sexo.
    """

    try:
        with open(caminho_pdf, 'rb') as arquivo:
            leitor_pdf = PyPDF2.PdfReader(arquivo)
            texto_completo = ""
            for pagina in leitor_pdf.pages:
                texto_completo += pagina.extract_text()

        texto_limpo = limpar_texto(texto_completo)

        dados_faixa_etaria = {
            '10 a 14 anos': {'Masculino': 0, 'Feminino': 0, 'Total': 0},
            '15 a 19 anos': {'Masculino': 0, 'Feminino': 0, 'Total': 0}
        }

        # Regex para encontrar a linha da faixa etária de 10 a 14 anos
        match_10_14 = re.search(r'10 a 14 anos\s+(\d+)\s+(\d+)\s+0\s+0\s+(\d+)', texto_limpo, re.IGNORECASE)
        if match_10_14:
            dados_faixa_etaria['10 a 14 anos']['Masculino'] = int(match_10_14.group(1))
            dados_faixa_etaria['10 a 14 anos']['Feminino'] = int(match_10_14.group(2))
            dados_faixa_etaria['10 a 14 anos']['Total'] = int(match_10_14.group(3))

        # Regex para encontrar a linha da faixa etária de 15 a 19 anos
        match_15_19 = re.search(r'15 a 19 anos\s+(\d+)\s+(\d+)\s+0\s+0\s+(\d+)', texto_limpo, re.IGNORECASE)
        if match_15_19:
            dados_faixa_etaria['15 a 19 anos']['Masculino'] = int(match_15_19.group(1))
            dados_faixa_etaria['15 a 19 anos']['Feminino'] = int(match_15_19.group(2))
            dados_faixa_etaria['15 a 19 anos']['Total'] = int(match_15_19.group(3))

        # Detectar tipo de relatório
        tipo_relatorio = "médico"
        if "relatório de atendimento odontológico" in texto_limpo.lower():
            tipo_relatorio = "odontológico"

        # Exibir os resultados
        print("=" * 60)
        print(f"ANÁLISE DE FAIXA ETÁRIA - RELATÓRIO {tipo_relatorio.upper()}")
        print("=" * 60)

        for faixa, dados in dados_faixa_etaria.items():
            print(f"\nFaixa Etária: {faixa}")
            print(f"  Masculino: {dados['Masculino']}")
            print(f"  Feminino: {dados['Feminino']}")
            print(f"  Total: {dados['Total']}")

        print("=" * 60)

        # Adicionar tipo de relatório ao resultado
        dados_faixa_etaria['tipo_relatorio'] = tipo_relatorio

        return dados_faixa_etaria

    except Exception as e:
        print(f"Erro ao processar o PDF: {str(e)}")
        return None

def processar_multiplos_pdfs():
    """
    Função para Google Colab que permite upload e análise de faixa etária de múltiplos PDFs
    Abre uma caixa de diálogo para selecionar arquivos PDF
    """
    print("📁 Selecione os arquivos PDF para análise de faixa etária...")
    print("(Você pode selecionar múltiplos arquivos segurando Ctrl/Cmd)")

    # Upload dos arquivos
    uploaded = files.upload()

    if not uploaded:
        print("❌ Nenhum arquivo foi enviado.")
        return None

    resultados = {}
    arquivos_pdf = []

    # Filtrar apenas arquivos PDF
    for filename in uploaded.keys():
        if filename.lower().endswith('.pdf'):
            arquivos_pdf.append(filename)
        else:
            print(f"⚠️  {filename} não é um arquivo PDF. Ignorando...")

    if not arquivos_pdf:
        print("❌ Nenhum arquivo PDF foi encontrado.")
        return None

    print(f"\n📊 Analisando faixa etária de {len(arquivos_pdf)} arquivo(s) PDF...")

    # Processar cada PDF
    for i, filename in enumerate(arquivos_pdf, 1):
        print(f"\n{'='*60}")
        print(f"PROCESSANDO ARQUIVO {i}/{len(arquivos_pdf)}: {filename}")
        print(f"{'='*60}")

        try:
            # Processar o PDF
            resultado = extrair_faixas_etarias_pdf(filename)
            resultados[filename] = resultado
            print(f"✅ {filename} processado com sucesso!")

        except Exception as e:
            print(f"❌ Erro ao processar {filename}: {str(e)}")
            resultados[filename] = None

        # Limpar o arquivo após processamento para economizar espaço
        try:
            os.remove(filename)
        except:
            pass

    # Resumo final
    print(f"\n🎉 Análise de faixa etária concluída!")
    print(f"📈 {len([r for r in resultados.values() if r is not None])} arquivo(s) analisado(s) com sucesso")
    print(f"❌ {len([r for r in resultados.values() if r is None])} arquivo(s) com erro")

    # Exibir resumo consolidado
    exibir_resumo_consolidado(resultados)

    return resultados

def exibir_resumo_consolidado(resultados):
    """
    Exibe um resumo consolidado de todos os PDFs analisados
    """
    print(f"\n{'='*80}")
    print("RESUMO CONSOLIDADO - ANÁLISE DE FAIXA ETÁRIA")
    print(f"{'='*80}")

    # Totais consolidados
    total_10_14_masc = 0
    total_10_14_fem = 0
    total_10_14_total = 0
    total_15_19_masc = 0
    total_15_19_fem = 0
    total_15_19_total = 0

    relatorios_medicos = 0
    relatorios_odontologicos = 0

    for arquivo, dados in resultados.items():
        if dados is not None:
            # Contabilizar tipos
            if dados.get('tipo_relatorio') == 'médico':
                relatorios_medicos += 1
            elif dados.get('tipo_relatorio') == 'odontológico':
                relatorios_odontologicos += 1

            # Somar totais
            total_10_14_masc += dados['10 a 14 anos']['Masculino']
            total_10_14_fem += dados['10 a 14 anos']['Feminino']
            total_10_14_total += dados['10 a 14 anos']['Total']

            total_15_19_masc += dados['15 a 19 anos']['Masculino']
            total_15_19_fem += dados['15 a 19 anos']['Feminino']
            total_15_19_total += dados['15 a 19 anos']['Total']

    print(f"📋 Relatórios Médicos: {relatorios_medicos}")
    print(f"🦷 Relatórios Odontológicos: {relatorios_odontologicos}")
    print(f"📊 Total de Relatórios: {relatorios_medicos + relatorios_odontologicos}")

    print(f"\n🧒 FAIXA ETÁRIA: 10 a 14 anos")
    print(f"   👦 Masculino: {total_10_14_masc}")
    print(f"   👧 Feminino: {total_10_14_fem}")
    print(f"   👥 Total: {total_10_14_total}")

    print(f"\n🧑 FAIXA ETÁRIA: 15 a 19 anos")
    print(f"   👨 Masculino: {total_15_19_masc}")
    print(f"   👩 Feminino: {total_15_19_fem}")
    print(f"   👥 Total: {total_15_19_total}")

    print(f"\n🎯 TOTAL GERAL ADOLESCENTES (10-19 anos)")
    print(f"   👥 Masculino: {total_10_14_masc + total_15_19_masc}")
    print(f"   👥 Feminino: {total_10_14_fem + total_15_19_fem}")
    print(f"   👥 Total: {total_10_14_total + total_15_19_total}")

    print(f"{'='*80}")

# Função alternativa para arquivos já no Colab
def processar_multiplos_pdfs_caminhos(caminhos_pdfs):
    """
    Processa múltiplos PDFs quando você já tem os caminhos dos arquivos

    Args:
        caminhos_pdfs (list): Lista com os caminhos dos arquivos PDF

    Returns:
        dict: Dicionário com os resultados de cada PDF
    """
    resultados = {}

    print(f"📊 Analisando faixa etária de {len(caminhos_pdfs)} arquivo(s) PDF...")

    for i, caminho in enumerate(caminhos_pdfs, 1):
        print(f"\n{'='*60}")
        print(f"PROCESSANDO ARQUIVO {i}/{len(caminhos_pdfs)}: {caminho}")
        print(f"{'='*60}")

        try:
            resultado = extrair_faixas_etarias_pdf(caminho)
            resultados[caminho] = resultado
            print(f"✅ {caminho} processado com sucesso!")

        except Exception as e:
            print(f"❌ Erro ao processar {caminho}: {str(e)}")
            resultados[caminho] = None

    print(f"\n🎉 Análise de faixa etária concluída!")
    print(f"📈 {len([r for r in resultados.values() if r is not None])} arquivo(s) analisado(s) com sucesso")
    print(f"❌ {len([r for r in resultados.values() if r is None])} arquivo(s) com erro")

    # Exibir resumo consolidado
    exibir_resumo_consolidado(resultados)

    return resultados

# Exemplo de uso
if __name__ == "__main__":
    print("🔧 SISTEMA DE ANÁLISE DE FAIXA ETÁRIA PARA MÚLTIPLOS PDFs")
    print("=" * 60)
    print("Funções disponíveis:")
    print("1. resultados = processar_multiplos_pdfs()              # Upload interativo")
    print("2. resultados = processar_multiplos_pdfs_caminhos(['arquivo1.pdf', 'arquivo2.pdf'])")
    print("3. resultado_unico = extrair_faixas_etarias_pdf('arquivo.pdf')  # PDF único")
    print("=" * 60)

🔧 SISTEMA DE ANÁLISE DE FAIXA ETÁRIA PARA MÚLTIPLOS PDFs
Funções disponíveis:
1. resultados = processar_multiplos_pdfs()              # Upload interativo
2. resultados = processar_multiplos_pdfs_caminhos(['arquivo1.pdf', 'arquivo2.pdf'])
3. resultado_unico = extrair_faixas_etarias_pdf('arquivo.pdf')  # PDF único


In [11]:
resultados = processar_multiplos_pdfs()

📁 Selecione os arquivos PDF para análise de faixa etária...
(Você pode selecionar múltiplos arquivos segurando Ctrl/Cmd)


Saving silvio 1.pdf to silvio 1 (1).pdf

📊 Analisando faixa etária de 1 arquivo(s) PDF...

PROCESSANDO ARQUIVO 1/1: silvio 1 (1).pdf
ANÁLISE DE FAIXA ETÁRIA - RELATÓRIO MÉDICO

Faixa Etária: 10 a 14 anos
  Masculino: 2
  Feminino: 1
  Total: 3

Faixa Etária: 15 a 19 anos
  Masculino: 0
  Feminino: 0
  Total: 0
✅ silvio 1 (1).pdf processado com sucesso!

🎉 Análise de faixa etária concluída!
📈 1 arquivo(s) analisado(s) com sucesso
❌ 0 arquivo(s) com erro

RESUMO CONSOLIDADO - ANÁLISE DE FAIXA ETÁRIA
📋 Relatórios Médicos: 1
🦷 Relatórios Odontológicos: 0
📊 Total de Relatórios: 1

🧒 FAIXA ETÁRIA: 10 a 14 anos
   👦 Masculino: 2
   👧 Feminino: 1
   👥 Total: 3

🧑 FAIXA ETÁRIA: 15 a 19 anos
   👨 Masculino: 0
   👩 Feminino: 0
   👥 Total: 0

🎯 TOTAL GERAL ADOLESCENTES (10-19 anos)
   👥 Masculino: 2
   👥 Feminino: 1
   👥 Total: 3
